In [1]:
import dai
import pandas as pd
import numpy as np
import time
from datetime import timedelta

In [2]:
ed = dai.query("SELECT DISTINCT date FROM cn_stock_bar1d WHERE date >= '2024-01-01'").df()['date'].max().strftime("%Y-%m-%d")
sd = (pd.Timestamp(ed) - timedelta(days=45)).strftime("%Y-%m-%d")

In [6]:
def get_factor_df_one_day(date):

    sql = f"""

    WITH 

    data1 AS (
        SELECT 
            *,
            make_date(year(date), month(date),  day(date))    AS day,
            make_time(hour(date), minute(date), second(date)) AS time,
            m_nanavg(volume, 5) / m_nanavg(volume, 30)        AS ATV,
            c_pct_rank(ATV)                                   AS rank_ATV,
        FROM cn_stock_bar1m
        WHERE date BETWEEN '{date} 00:00:00.000' AND '{date} 23:59:59.999'
        AND instrument NOT LIKE '%BJ'
        ORDER BY date, instrument
    ),

    data2 AS (
        SELECT
            *,
            day,
            time,
            AVG(rank_ATV) OVER(PARTITION BY day, instrument)      AS mean_rank_ATV,
            STDDEV(rank_ATV) OVER(PARTITION BY day, instrument)   AS std_rank_ATV,
            KURTOSIS(rank_ATV) OVER(PARTITION BY day, instrument) AS kurtosis_rank_ATV,
        FROM data1
        ORDER BY instrument, date
    ),

    data3 AS (
        SELECT DISTINCT
            day AS date,
            instrument,
            mean_rank_ATV,
            std_rank_ATV,
            kurtosis_rank_ATV,
        FROM data2
    ),

    data4 AS (
        SELECT
            date,
            instrument,
            mean_rank_ATV                                    AS alpha_hf_00010001,
            std_rank_ATV                                     AS alpha_hf_00010002,
            kurtosis_rank_ATV                                AS alpha_hf_00010003,
            mean_rank_ATV / std_rank_ATV + kurtosis_rank_ATV AS alpha_hf_00010004,
        FROM data3
    )

    SELECT * 
    FROM  data4
    
    QUALIFY COLUMNS(*) IS NOT NULL
    ORDER BY date, instrument
    """

    return dai.query(sql).df().dropna()

In [7]:
get_factor_df_one_day(ed)

,date,instrument,alpha_hf_00010001,alpha_hf_00010002,alpha_hf_00010003,alpha_hf_00010004
0,2026-02-13,000001.SZ,0.513769,0.252115,-1.114597,0.923237
1,2026-02-13,000002.SZ,0.545397,0.258774,-0.998168,1.109450
2,2026-02-13,000004.SZ,0.359238,0.306814,-0.926551,0.244318
3,2026-02-13,000006.SZ,0.523355,0.270447,-1.139294,0.795858
4,2026-02-13,000007.SZ,0.480149,0.338889,-1.522113,-0.105282
...,...,...,...,...,...,...
5175,2026-02-13,688816.SH,0.483892,0.289324,-1.307248,0.365240
5176,2026-02-13,688818.SH,0.525583,0.252966,-1.114551,0.963135
5177,2026-02-13,688819.SH,0.540196,0.242235,-0.876102,1.353948
5178,2026-02-13,688981.SH,0.545103,0.201891,-0.735125,1.964857
